# Muestra básica de AutoGen

En este ejemplo de código, utilizará el [AutoGen](https://aka.ms/ai-agents/autogen) Marco de IA para crear un agente básico.

El objetivo de este ejemplo es mostrarle los pasos que usaremos más adelante en los ejemplos de código adicionales al implementar los diferentes patrones de agencia.

## Import the Needed Python Packages 

In [5]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## Create the Client 

En este ejemplo, utilizaremos[GitHub Models](https://aka.ms/ai-agents-beginners/github-models) para acceder al LLM. 

El `model` esta definidocomo  `gpt-4o-mini`. TPrueba a cambiarlo por otro disponible en la tienda de modelos de GitHub para ver los diferentes resultados.

Como prueba rápida, simplemente ejecutaremos un mensaje simple. - `¿Cuál es la capital de Francia?`. 

In [7]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="¿Cuál es la capital de Francia?", source="user")])
print(result)

ServiceRequestError: Cannot connect to host models.inference.ai.azure.com:443 ssl:default [getaddrinfo failed]

## Definición del agente

Ahora que hemos configurado el `client` y confirmado que funciona, vamos a crear un `AssistantAgent`. EA cada agente se le puede asignar: 
**name** - Un nombre abreviado que será útil para referenciarlo en flujos de múltiples agentes.
**model_client** - El cliente que creó en el paso anterior.
**tools** - Herramientas disponibles que el Agente puede utilizar para completar una tarea.
**system_message** - El metaprompt que define la tarea, el comportamiento y el tono del LLM.

Puedes cambiar el mensaje del sistema para ver cómo responde el LLM. Abordaremos este tema. `tools` en la lección #4. 


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="Eres un agente de viajes que planifica unas vacaciones fantásticas.",
)

## Ejecutar el agente

La siguiente función ejecutará el agente. Usamos el `on_message` método para actualizar el estado del Agente con el nuevo mensaje. 

En este caso, actualizamos el estado con un nuevo mensaje del usuario que es `"Planifícame unas estupendas vacaciones soleadas"`.

Puede cambiar el contenido del mensaje para ver cómo el LLM responde de manera diferente.

In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Planifícame unas estupendas vacaciones soleadas"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()